# Let's generate some music!

1. Run through the basic set up code blocks.
2. Load the model, reconstruct or generate new music.

## Basic Setup

In [11]:
import os
import torch
import numpy as np
import torch.nn as nn
import muse.supplier as spr
import muse.processor2 as pcr
import muse.model2 as mdl
import muse.trainer as trn
import muse.visualizer as vis

import warnings
warnings.filterwarnings("ignore")

In [12]:
### ===== Hyperparameters ===== ###

instrument = 'Piano'
filepath = "./../input/haydn/"
song_len = 200*4
stride = 200
device = trn.get_device()

seed_load = 592643464

In [13]:
### ===== Data Preprocessing ===== ###

all_midis, filenames = pcr.get_midis(filepath) # load all .midi files
Corpus, instru2corpus = pcr.get_notes_batch(all_midis) # extract all notes and sort by instrument
Corpus, fmap, rmap = pcr.get_map(Corpus) # get forward-map and reverse-map from corpus
instru2corpus = pcr.remove_short(instru2corpus) # remove songs that are too short
instru2corpus = pcr.upsample_batch(instru2corpus) # upsample notes according to multiples of duration

X_train_melody, X_val_melody, X_train_offset, X_val_offset = pcr.train_test_split(instru2corpus, instrument, fmap, song_len, stride,\
                                                                                  seed=seed_load, process='center')
X_train_melody, X_val_melody = pcr.batchify(X_train_melody), pcr.batchify(X_val_melody) # reshape and turn into tensor
X_train_offset, X_val_offset = pcr.batchify(X_train_offset), pcr.batchify(X_val_offset) # reshape and turn into tensor

classes = len(set(Corpus)) # get number of unique keys

## Variational Autoencoder

In [14]:
### ===== Load model ===== ###

model1 = mdl.cnn_varautoencoder(1, 4, 61, std=1.0)
model4_mel = trn.load_model('vae3_melody_CE', model1, device)[0]
model4_mel.eval()

cnn_varautoencoder(
  (encoder): cnn_varencoder(
    (conv1): Conv1d(1, 16, kernel_size=(21,), stride=(1,))
    (conv21): Conv1d(16, 4, kernel_size=(11,), stride=(1,))
    (conv22): Conv1d(16, 4, kernel_size=(11,), stride=(1,))
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (relu): ReLU()
    (flat): Flatten(start_dim=1, end_dim=-1)
    (linear): Linear(in_features=760, out_features=20, bias=True)
  )
  (decoder): cnn_vardecoder(
    (tconv1): ConvTranspose1d(4, 16, kernel_size=(11,), stride=(1,))
    (tconv2): ConvTranspose1d(16, 1, kernel_size=(31,), stride=(1,))
    (relu): ReLU()
    (sigmoid): Sigmoid()
    (linear): Linear(in_features=20, out_features=3040, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
)

In [10]:
### ===== Reconstruction ===== ###

for idx in range(len(X_train_melody)):
    pcr.gen_reconstruction(model4_mel, None, X_train_melody, X_train_offset, rmap, base=1, idx=idx) # idx = 80, 60

In [15]:
### ===== Generation ===== ###

mus = []
for song in X_train_melody:
    mu, logvar = model4_mel.encoder(song)
    mus.append(mu.view(-1).detach().numpy())

mu_avg = np.mean(np.array(mus), axis=0)
cov_avg = np.cov(np.array(mus), rowvar=False)

for i in range(20):
    _ = pcr.gen_generation_vae(mu_avg, cov_avg, model4_mel, rmap, fname='new_'+str(i))